testing notebook

In [ ]:
import torch
from minito.core import cfm,dataloaders
from minito.models.vector_fields import UltraVectorField
import wandb
import lightning as pl
DEVICE = 'cpu'

In [ ]:

ds= dataloaders.LaggedNumpyDataInMemory("/Users/simonols/Downloads/DW.npz", max_lag=200)
data_loader=torch.utils.data.DataLoader(ds, batch_size=64, shuffle=True)

In [ ]:
vf = UltraVectorField()
model = cfm.CFM(vf, lr=1e-4)


In [ ]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from datetime import timedelta
import wandb


wandb_logger = WandbLogger(project="minito", log_model="all")
DEVICE='cpu'

monitor = "train/loss"
model_callback = ModelCheckpoint(
    monitor=monitor, 
    filename="DW-{epoch:02d}-{step}",
    save_top_k=3,            # Save only the best 3 models
    mode="min",               # We want to minimize training loss
    every_n_train_steps=None, # Set this if you want to check more often than once per epoch
    train_time_interval=timedelta(minutes=5),
)

trainer = pl.Trainer(
    max_epochs=10,
    accelerator=DEVICE,
    devices="auto",
    logger=wandb_logger,     # Add logger here
    callbacks=[model_callback]
)

trainer.fit(model, data_loader)

In [ ]:
batch = data_loader.__iter__()


In [ ]:
def make_batch_for_sampling(data_loader, base_distribution, lag=100, beta=1):
    batch = next(data_loader.__iter__())
    batch['corr'] = {k: v.clone() if isinstance(v, torch.Tensor) else v for k, v in batch['cond'].items()} 
    batch['corr']['x'] = base_distribution.sample_as(batch['cond']['x'])
    batch['lag']=torch.ones_like(batch['lag'])*lag
    batch['cond']['beta']=torch.ones_like(batch['cond']['beta'])*beta

    return batch



In [ ]:
b=make_batch_for_sampling(data_loader, dataloaders.BaseDensity(1.0))


In [ ]:
a=model.sample(b, ode_steps=50, nested_samples=5000)

In [ ]:
class WolfeQuappEnergy:
    def __init__(self):
        self.dim = 2
        pass

    def energy(self, x):
        return x[0]**4+x[1]**4-2*x[0]**2-4*x[1]**2+x[0]*x[1]+0.3*x[0]+0.1*x[1]

In [ ]:
import matplotlib.pyplot as plt
# xmin, xmax = -2, 2
# ymin, ymax = -2, 2
# npoints = 100
# x = torch.linspace(xmin, xmax, npoints)
# y = torch.linspace(ymin, ymax, npoints)
# X, Y = torch.meshgrid(x, y)
# pos = torch.stack([X, Y], dim=-1)
# wq_energy = WolfeQuappEnergy()
# Z = wq_energy.energy(pos.reshape(-1,2).T).reshape(npoints, npoints)
# plt.contourf(X.numpy(), Y.numpy(), Z.numpy(), levels=50, cmap='viridis')
for t in a['traj']['x'][1:,:,:].transpose(1,0):
    plt.plot(t[:,0].cpu())

In [ ]:
import numpy as np
plt.hist(a['traj']['x'].flatten().numpy(), bins=np.linspace(-2,2,100))    

In [ ]:
plt.hist(ds.x_data[ds.beta_data==1].squeeze().numpy(), bins=np.linspace(-2,2,100));

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('psolsson/minito/model-8lhcrof0:v3', type='model')
artifact_dir = artifact.download()

In [ ]:
import torch
from minito.core import cfm,dataloaders
import wandb
import lightning as pl
DEVICE = 'cpu'

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('psolsson/minito/model-8lhcrof0:v3', type='model')
artifact_dir = artifact.download()

In [ ]:
cfm_model = cfm.CFM.load_from_checkpoint(f"{artifact_dir}/model.ckpt")

In [ ]:
def make_batch_for_sampling(data_loader, base_distribution, lag=10, beta=1.0):
    batch = next(data_loader.__iter__())
    batch['corr'] = {k: v.clone() if isinstance(v, torch.Tensor) else v for k, v in batch['cond'].items()} 
    batch['corr']['x'] = base_distribution.sample_as(batch['cond']['x'])
    batch['lag']=torch.ones_like(batch['lag'])*lag
    batch['cond']['beta']=torch.ones_like(batch['cond']['beta'])*beta

    return batch



In [ ]:
b=make_batch_for_sampling(data_loader, dataloaders.BaseDensity(1.0))


In [ ]:
a=cfm_model.sample(b, ode_steps=50, nested_samples=1000)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
import numpy as np
plt.hist(a['traj_x'].flatten().numpy(), bins=np.linspace(-2,2,100))    

In [ ]:
plt.plot(a['traj_x'][:,10,0])